##### Data Visualization
每十天划分一段数据，勾画61天共6个热力图，每个热力图使用数据：点坐标，该点在某时间段内阳性人员数量

In [1]:
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
files_path = 'C:/Users/Deng_/Desktop/datas/'
acid_sample_info = pd.read_csv(files_path+'acid_sample_info.csv')
venue_info = pd.read_csv(files_path+'venue_info.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Deng_/Desktop/datas/acid_sample_info.csv'

In [ ]:
import pymysql

db = pymysql.connect(
    host='localhost',
    user='root',
    password='ff2017',
    charset='utf8',
    database='tedi_data'
)

##### 通过sql快速获取阳性人员的出行信息

In [ ]:
## 获取阳性人员的出行数据
q = '''
select * from pos_travel_info'''
pos_travel_info = pd.read_sql(q, con=db)
pos_travel_info = pos_travel_info[['date','grid_id']]
pos_travel_info

##### 切片获取每一段时间的出行数据

In [ ]:
sliced_travel_info = []
cur_time = pd.to_datetime('2022-09-30',format='%Y-%m-%d')
end = pd.to_datetime('2022-11-30',format='%Y-%m-%d')
time_list = []
while cur_time < end:
    this_end = cur_time + pd.Timedelta(days=10)
    if this_end == pd.to_datetime('2022-10-30',format='%Y-%m-%d') or this_end == pd.to_datetime('2022-11-30',format='%Y-%m-%d'): this_end += pd.Timedelta(days=1)
    time_list.append(this_end)

    mask = (pos_travel_info['date']>=cur_time) & (pos_travel_info['date']<=this_end)
    this = pos_travel_info[mask]
    sliced_travel_info.append(this)
    cur_time = this_end

##### 获取分段后出行信息点坐标

In [ ]:
### 定义函数： 给定key-->list，建立一个所有键值为 0 的字典
def todict(x):
    my_set = x
    my_dict = {key: 0 for key in my_set}
    return my_dict


grids_dict = todict(venue_info.grid_point_id.tolist())
grids_dict

##### 利用分段数据，获取每一段中每个地点的阳性人数

In [ ]:
grids_dicts = []
for each in sliced_travel_info:
    grids_dict = todict(venue_info.grid_point_id.tolist()) ## 每读取一段时间数据，清空现有数据
    for i in each.index:
        gid = each['grid_id'][i]
        grids_dict[gid] += 1
    grids_dicts.append(grids_dict)
grids_dicts[1]

In [ ]:
sliced_grid_info = []
for each in sliced_travel_info:
    this = pd.merge(left = each, right = venue_info, left_on='grid_id', right_on='grid_point_id',how = 'left')
    this = this[['grid_id','x_coordinate','y_coordinate']]
    sliced_grid_info.append(this)

##### 数据获取完毕，确保三者 shape 一致

In [ ]:
len(sliced_grid_info)==len(sliced_travel_info)==len(grids_dicts)

In [ ]:
## 点坐标
sliced_grid_info[0]

In [ ]:
## 点人数
grids_dicts[0]

##### 合并两个df用于绘图

In [ ]:
for i in range(6):
    sliced_grid_info[i]['num'] = 0
    for idx in sliced_grid_info[i].index:
        gid = sliced_grid_info[i]['grid_id'][idx]
        sliced_grid_info[i]['num'][idx] = grids_dicts[i][gid]

In [ ]:
print(sliced_grid_info[0])

##### 开始绘制每个时间段的热力图

In [ ]:
sliced_grid_info[0]

In [ ]:
time_list

In [ ]:
%matplotlib inline
def color_map(datas):
    re = []
    for n in datas:
        s = 0
        if n<50: s = (0.55,0.7,0.97)
        elif 50<=n<100: s = (0.44,0.5,0.7)
        elif 100<=n<150: s =(0.55,0.3,0.61)
        elif 150<=n<200: s =(0.62,0.2,0.5)
        elif 200<=n<250: s =(0.71,0.1,0.41)
        elif 250<=n<300: s =(0.83,0.05,0.302)
        else : s =(1,0,0)
        re.append(s)
    return re

def save_plot(dff, idd):
    plt.figure(figsize=(50,40))
    s = ''
    if idd==0: s = "10-01 --- "+str(time_list[i])[5:10]
    else : s = str(time_list[i-1])[5:10] + " ---" + str(time_list[i])[5:10]
    plt.title(s)
    x = dff.x_coordinate.to_list()
    y = dff.y_coordinate.to_list()
    datas = dff.num.to_list()
    plt.scatter(x,y, marker='o', color = color_map(datas),alpha=0.5)
    plt.show()
    plt.savefig('C:/Users/Deng_/PycharmProjects/test_for_tedi/tedi-230412/##{}.png'.format(s))

def get_pics(List):
    for idd in range(6):
        df = List[i]
        save_plot(df,idd)

In [ ]:
get_pics(sliced_grid_info)